## UAS PMDM 

In [2]:
import os
import re
import sys
import string
import modSpellChecker_1 as sc
from contractions_1 import CONTRACTION_MAP
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import pandas as pd
import operator

In [3]:
character = ['z','y','x','w','v','u','t','s','r','q','p','o','n','m','l','k','j','i','h','g','f','e','d',',',';',':','-','...','?','!', '(',')','[',']','{','}','<','>', '"','/','\'','#','-','@']

def repeatcharNormalize(text):
    for i in range(len(character)):
        charac_long = 5
        while charac_long >= 2:
            char = character[i] * charac_long
            text = text.replace(char, character[i])
            charac_long -= 1
    return text

def spellNormalize(text):
    spellCheck = []
    for i in text:
        if i not in character:
            j = sc.correction(i)
            spellCheck.append(j)
        else:
            spellCheck.append(i)
    return spellCheck

def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def stemmer_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)
    return text

In [4]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('',token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [5]:
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

In [6]:
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [7]:
def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = stemmer_text(text)
        text = remove_special_characters(text)
        text = repeatcharNormalize(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            text = spellNormalize(text)
            normalized_corpus.append(text)
    return normalized_corpus

Penyiapan Data Teks

In [8]:
dataset = pd.read_csv('cnn_sport.csv')
dataset

,title,category,link
0,Klasemen Liga Inggris: Tottenham Gusur Man Cit...,Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
1,Khamzat Disebut Tak Pantas Dapat Duel Gelar Ju...,Olahraga Lainnya,https://www.cnnindonesia.com/olahraga/20231023...
2,"Hasil Liga Inggris: Son Cetak Gol, Tottenham H...",Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
3,Binder Merasa Kasihan Martin Gagal Menang di M...,Moto GP,https://www.cnnindonesia.com/olahraga/20231024...
4,FOTO: Tottenham Jaga Rekor Belum Terkalahkan U...,Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
...,...,...,...
1995,Jadwal Siaran Langsung Final Hong Kong Open: 3...,Raket,https://www.cnnindonesia.com/olahraga/20230917...
1996,Klasemen Liga Inggris Usai MU Terpuruk di Old ...,Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...
1997,Ronaldo Tunjukkan Gairah Besar di Saudi Pro Le...,Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...
1998,"2 Pemain Al Raed Berebut Tendang Penalti, Rona...",Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...


In [9]:
feature = dataset.iloc[:,0]

In [10]:
feature[0:10]

0    Klasemen Liga Inggris: Tottenham Gusur Man Cit...
1    Khamzat Disebut Tak Pantas Dapat Duel Gelar Ju...
2    Hasil Liga Inggris: Son Cetak Gol, Tottenham H...
3    Binder Merasa Kasihan Martin Gagal Menang di M...
4    FOTO: Tottenham Jaga Rekor Belum Terkalahkan U...
5    INFOGRAFIS: Jadwal Timnas Indonesia di Kualifi...
6          Messi Banggakan Capaian Bersama Inter Miami
7    Erick Thohir Soal Persiapan Piala Dunia U-17: ...
8    Bagnaia Mulai Pongah Jelang MotoGP Thailand da...
9    Sidang Komdis PSSI: Hugo Samir Pukul Pemain Pe...
Name: title, dtype: object

In [11]:
norm_corpus = normalize_corpus(feature)
len(norm_corpus)

2000

Ekstraksi Fitur dengan TFIDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(2000, 2862)

Metode Cosine Similarity

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1.000000,0.0,0.196113,0.0,0.081744,0.0,0.0,0.0,0.000000,0.0,...,0.321987,0.0,0.0,0.000000,0.0,0.0,0.255013,0.0,0.0,0.028092
1,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0.196113,0.0,1.000000,0.0,0.363310,0.0,0.0,0.0,0.000000,0.0,...,0.219156,0.0,0.0,0.135407,0.0,0.0,0.126328,0.0,0.0,0.224574
3,0.000000,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0,0.032944,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,0.081744,0.0,0.363310,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,...,0.055893,0.0,0.0,0.000000,0.0,0.0,0.030026,0.0,0.0,0.000000


In [14]:
article_list = dataset['title'].values
article_list, article_list.shape

(array(['Klasemen Liga Inggris: Tottenham Gusur Man City dari Puncak',
        'Khamzat Disebut Tak Pantas Dapat Duel Gelar Juara UFC',
        'Hasil Liga Inggris: Son Cetak Gol, Tottenham Hajar Fulham 2-0',
        ...,
        'Ronaldo Tunjukkan Gairah Besar di Saudi Pro League: Marah dan Gembira',
        '2 Pemain Al Raed Berebut Tendang Penalti, Ronaldo Ikut Campur',
        'Hasil Liga Spanyol: Joao Felix Cetak Gol, Barcelona Bungkam Betis 5-0'],
       dtype=object),
 (2000,))

In [43]:
article_idx = np.where(article_list=='Klasemen Liga Inggris: Tottenham Gusur Man City dari Puncak')[0][0]
article_idx

0

In [44]:
article_similarities = doc_sim_df.iloc[article_idx].values
article_similarities

array([1.        , 0.        , 0.19611333, ..., 0.        , 0.        ,
       0.02809224])

In [47]:
similar_article_idxs = np.argsort(-article_similarities)[1:10]
similar_article_idxs

array([  89,  845, 1476, 1212,  847, 1623, 1211, 1386,   52])

In [48]:
similar_article = article_list[similar_article_idxs]
similar_article

array(['Top 3 Sports: Tottenham Gusur Man City, Makhachev Sehebat Khabib',
       'Klasemen Liga Inggris Usai Arsenal Gebuk Man City',
       'Klasemen Liga Inggris: Brighton ke Tiga Besar, Arsenal Tergusur',
       'Hasil Liga Inggris: Bantai Bournemouth, Arsenal Tempel Man City',
       'Hasil Liga Inggris: Arsenal Benamkan Man City 1-0',
       'Hasil Liga Inggris: 10 Pemain Man City Bekuk Nottingham 2-0',
       'Hasil Liga Inggris: Man City dan MU Kompak Kalah',
       'FOTO: Man City Tumbang di Piala Liga',
       'Hasil Liga Inggris: Man City Tekuk Brighton 2-1, Haaland Cetak Gol'],
      dtype=object)

In [49]:
def article_recommender(article_title, articles=article_list,doc_sims=doc_sim_df):
    article_idx = np.where(articles == article_title)[0][0]
    article_similarities = doc_sims.iloc[article_idx].values
    similar_article_idxs = np.argsort(-article_similarities)[1:10]
    similar_articles = articles[similar_article_idxs]
    return similar_articles

In [50]:
popular_articles = ['Klasemen Liga Inggris: Tottenham Gusur Man City dari Puncak',
]

In [51]:
for article in popular_articles:
    print('Article:', article)
    print('Top 5 recommended Article:',article_recommender(article_title=article))
    print()

Article: Klasemen Liga Inggris: Tottenham Gusur Man City dari Puncak
Top 5 recommended Article: ['Top 3 Sports: Tottenham Gusur Man City, Makhachev Sehebat Khabib'
 'Klasemen Liga Inggris Usai Arsenal Gebuk Man City'
 'Klasemen Liga Inggris: Brighton ke Tiga Besar, Arsenal Tergusur'
 'Hasil Liga Inggris: Bantai Bournemouth, Arsenal Tempel Man City'
 'Hasil Liga Inggris: Arsenal Benamkan Man City 1-0'
 'Hasil Liga Inggris: 10 Pemain Man City Bekuk Nottingham 2-0'
 'Hasil Liga Inggris: Man City dan MU Kompak Kalah'
 'FOTO: Man City Tumbang di Piala Liga'
 'Hasil Liga Inggris: Man City Tekuk Brighton 2-1, Haaland Cetak Gol']



Metode BM25 Similarity

In [36]:
"""
Data:
-----
.. data:: PARAM_K1 - Free smoothing parameter for BM25.
.. data:: PARAM_B - Free smoothing parameter for BM25.
.. data:: EPSILON - Constant used for negative idf of document in corpus.
"""

import math
from six import iteritems
from six.moves import xrange

PARAM_K1 = 2.5
PARAM_B = 0.85
EPSILON = 0.2

class BM25(object):
    """Implementation of Best Matching 25 ranking function.
    Attributes
    ----------
    corpus_size : int
        Size of corpus (number of documents).
    avgdl : float
        Average length of document in `corpus`.
    corpus : list of list of str
        Corpus of documents.
    f : list of dicts of int
        Dictionary with terms frequencies for each document in `corpus`.
    df : dict
        Dictionary with terms frequencies for whole `corpus`.
    idf : dict
        Dictionary with inversed terms frequencies for while `corpus`.
    doc_len : list of int
        List of document lengths.
    """

    def __init__(self, corpus):
        """
        Parameters
        ----------
        corpus : list of list of str
            Given corpus.
        """
        self.corpus_size = len(corpus)
        self.avgdl = sum(float(len(x)) for x in corpus) / self.corpus_size
        self.corpus = corpus
        self.f = []
        self.df = {}
        self.idf = {}
        self.doc_len = []
        self.initialize()

    def initialize(self):
        for document in self.corpus:
            frequencies = {}
            self.doc_len.append(len(document))
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.f.append(frequencies)

            for word, freq in iteritems(frequencies):
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1

        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq)

    def get_score(self, document, index, average_idf):
        """Computes BM25 score of given `document` in relation to item of corpus
        Parameters
        ----------
        document : list of str
            Document to be scored.
        index : int
            Index of document in corpus selected to score with `document`.
        average_idf : float
            Average idf in corpus.
        Returns
        -------
        float
            BM25 score.
        """
        score = 0
        for word in document:
            if word not in self.f[index]:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += (idf * self.f[index][word] * (PARAM_K1 + 1)) / (self.f[index][word] + 
                                                                     PARAM_K1 * (1 - PARAM_B + PARAM_B *self.doc_len[index] / self.avgdl))
        return score

    def get_scores(self, document, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(document, index, average_idf)
            scores.append(score)
        return scores


def get_bm25_weights(corpus):
    """Returns BM25 scores (weights) of documents in corpus.
    Each document has to be weighted with every document in given corpus.
    Parameters
    ----------
    corpus : list of list of str
        Corpus of documents.
    Returns
    -------
    list of list of float
        BM25 scores.
    Examples
    --------
    >>> from gensim.summarization.bm25 import get_bm25_weights
    >>> corpus = [
    ...     ["black", "cat", "white", "cat"],
    ...     ["cat", "outer", "space"],
    ...     ["wag", "dog"]
    ... ]
    >>> result = get_bm25_weights(corpus)
    """
    bm25 = BM25(corpus)
    average_idf = sum(float(val) for val in bm25.idf.values()) / len(bm25.idf)

    weights = []
    for doc in corpus:
        scores = bm25.get_scores(doc, average_idf)
        weights.append(scores)

    return weights

In [37]:
a = [nltk.word_tokenize(doc) for doc in norm_corpus]

In [38]:
norm_corpus_tokens = np.asarray(a, dtype="object")

In [39]:
norm_corpus_tokens[:3]

array([list(['klasemen', 'liga', 'ingris', 'totenham', 'gusur', 'man', 'city', 'puncak']),
       list(['khamzat', 'sebut', 'tak', 'pantas', 'duel', 'gelar', 'juara', 'ufc']),
       list(['hasil', 'liga', 'ingris', 'son', 'cetak', 'gol', 'totenham', 'hajar', 'fulham', '20'])],
      dtype=object)

In [40]:
%%time
wts = get_bm25_weights(norm_corpus_tokens)

CPU times: user 2.96 s, sys: 36 ms, total: 3 s
Wall time: 3 s


In [41]:
bm25_wts_df = pd.DataFrame(wts)
bm25_wts_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,34.876276,0.000000,9.826123,0.000000,4.986329,0.0,0.0,0.0,0.000000,0.0,...,12.654752,0.0,0.0,0.000000,0.0,0.0,9.533070,0.0,0.0,2.016136
1,0.000000,39.861775,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,11.284396,0.000000,36.953930,0.000000,15.521029,0.0,0.0,0.0,0.000000,0.0,...,10.209499,0.0,0.0,5.778618,0.0,0.0,6.298067,0.0,0.0,9.492777
3,0.000000,0.000000,0.000000,36.320898,0.000000,0.0,0.0,0.0,2.094916,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,4.986329,0.000000,13.515259,0.000000,35.906775,0.0,0.0,0.0,0.000000,0.0,...,3.360153,0.0,0.0,0.000000,0.0,0.0,2.464951,0.0,0.0,0.000000


In [42]:
for article in popular_articles:
    print('Article:', article)
    print('Top 5 recommended Articles:',article_recommender(article_title=article, doc_sims=bm25_wts_df))
    print()

Article: Klasemen Liga Inggris: Tottenham Gusur Man City dari Puncak
Top 5 recommended Articles: ['Klasemen Liga Inggris Usai Arsenal Gebuk Man City'
 'Top 3 Sports: Tottenham Gusur Man City, Makhachev Sehebat Khabib'
 'Hasil Liga Inggris: Man City dan MU Kompak Kalah'
 'Hasil Liga Inggris: Arsenal Benamkan Man City 1-0'
 'Klasemen Liga Inggris: Brighton ke Tiga Besar, Arsenal Tergusur']

